# Intro to gragh convolutional surrogate models
Eamon Whalen

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import altair as alt

from gcnSurrogate.models.feastnetSurrogateModel import FeaStNet
from gcnSurrogate.readers.loadConmechGraphs import loadConmechGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
# dataDir = 'data/2D_Truss_v1.3/conmech/design_7_N_1000/'
# allGraphsUnfiltered = loadConmechGraphs(dataDir)

# dataDir = 'data/tower1.0/conmech/design_5_N_1000/'
# allGraphsUnfiltered = loadConmechGraphs(dataDir, loadDims=[0])

dataDir = 'data/djmmBridge1.0/conmech/bridge_N_1000/'
allGraphsUnfiltered = loadConmechGraphs(dataDir)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.059758
std,0.012355
min,0.045266
25%,0.051543
50%,0.055906
75%,0.063586
max,0.124130


In [3]:
# tower
# plotTruss(allGraphsUnfiltered[0], 
#           showDeformed=True, 
#           defScale=100, 
#           width=80, 
#           lineWidth=1, 
#           showLoads=True, 
#           showSupports=True, 
#           loadTheta=0, 
#           loadSize=100, 
#           supportSize=100)

# bridge
plotTruss(allGraphsUnfiltered[0], 
          showDeformed=True, 
          defScale=300, 
          width=500, 
          lineWidth=0.7, 
          showLoads=True, 
          showSupports=True, 
          loadSize=100, 
          supportSize=150)

alt.LayerChart(...)

## 2. Filter and partition

In [4]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.056490
std,0.006821
min,0.045266
25%,0.051163
50%,0.055148
75%,0.060596
max,0.075970


## 3. Train a GCN

In [5]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                         epochs=100,
                         batchSize=32,
                         saveDir='results/gcnBridge01/')

plotHistory(history)

epoch: 0   trainLoss: 1.9441e-01   valLoss:1.4346e-01  time: 9.00e+00
epoch: 1   trainLoss: 2.9341e-02   valLoss:3.9328e-02  time: 8.76e+00
epoch: 2   trainLoss: 2.5169e-02   valLoss:3.0119e-02  time: 8.74e+00
epoch: 3   trainLoss: 1.8760e-02   valLoss:5.0938e-02  time: 8.81e+00
epoch: 4   trainLoss: 1.7755e-02   valLoss:3.0999e-02  time: 8.81e+00
epoch: 5   trainLoss: 1.4585e-02   valLoss:3.6146e-02  time: 8.80e+00
epoch: 6   trainLoss: 1.2230e-02   valLoss:5.9460e-02  time: 8.79e+00
epoch: 7   trainLoss: 1.3504e-02   valLoss:1.5232e-02  time: 8.80e+00
epoch: 8   trainLoss: 1.0827e-02   valLoss:2.1540e-02  time: 8.72e+00
epoch: 9   trainLoss: 1.0660e-02   valLoss:1.8295e-02  time: 8.79e+00
epoch: 10   trainLoss: 1.2923e-02   valLoss:2.2606e-02  time: 8.76e+00
epoch: 11   trainLoss: 1.2364e-02   valLoss:1.9020e-02  time: 8.81e+00
epoch: 12   trainLoss: 1.5288e-02   valLoss:1.9021e-02  time: 8.79e+00
epoch: 13   trainLoss: 1.0467e-02   valLoss:1.6458e-02  time: 8.76e+00
epoch: 14   trai

alt.Chart(...)

## 4. Test the GCN

In [6]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000008,0.002143,0.070155,0.968620,0.975047,0.253176,-16.498466
test,0.000008,0.002150,0.069851,0.967523,0.975920,0.201861,-16.150501


## 5. Visualize some predictions

In [10]:
i = 4
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=500, prediction=pred, lineWidth=0.7)

alt.LayerChart(...)

In [8]:
# # interactive scatter plot
# alt.data_transformers.enable('json')
# allPreds = gcn.predict(testData)
# display(interactiveErrorPlot(testData, allPreds))
# alt.data_transformers.enable('default');